### C# Interactive Notebook - Carbon Example No.3

This example is a slightly more spohisticated version of C# example notebook no.2. A cross-tabulation report of brand popularity by month over a 5 year period is converted into a Microsoft DataFrame which is input to [Plotly.net](https://plotly.net/reference/index.html) to create a line chart.

In [100]:
#r "nuget: RCS.Carbon.Tables"
#r "nuget: DataView.InteractiveExtension,*-*"
#r "nuget: Microsoft.ML.DataView"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Data"

using RCS.Carbon.Tables;
using RCS.Carbon.Variables;
using RCS.Carbon.Shared;

Installed Packages DataView.InteractiveExtension, 1.0.69 FSharp.Data, 5.0.2 Microsoft.Data.Analysis, 0.19.1 Microsoft.ML.DataView, 1.7.1 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2 RCS.Carbon.Tables, 8.3.19

In [107]:

var engine = new CrossTabEngine();
await engine.LoginId("16499372", "C6H12O6");
engine.OpenJob("rcsruby", "demo");

var sprops  = new XSpecProperties(); 
sprops.InitAsMissing = true;

var dprops  = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.TSV;
dprops.Cells.Frequencies.Visible = false;
dprops.Cells.RowPercents.Visible = true; // need this as Carbon table is transposed
dprops.Cells.ColumnPercents.Visible = false;
dprops.Decimals.Frequencies = 0;
dprops.Decimals.Expressions = 2;
dprops.Decimals.Statistics = 2;

dprops.Titles.Top.Visible = false;
dprops.Titles.Side.Visible = false;
dprops.Titles.Filter.Visible = false;
dprops.Titles.Weight.Visible = false;
dprops.Titles.Status.Visible = false;
dprops.Titles.Name.Visible = false;

dprops.Corner.Priority = XPriority.Top;

string chartXaxis ="month";
string seriesPlots = "uba";

string ret = engine.GenTab("Example 04", seriesPlots, chartXaxis + "(*)", null, null, sprops, dprops);  //.ToArray<String>();

string strRet = String.Join(Environment.NewLine, ret).Replace('*','0');

Console.WriteLine(chartXaxis + strRet);

month	Brand1	Brand2	Brand3	Brand4	Brand5	Brand6	Brand7	Brand8	Brand9	Brand10
Jan2011	68.00%	44.00%	57.00%	49.00%	35.00%	25.50%	18.00%	9.50%	8.50%	14.00%
Feb2011	65.00%	47.00%	59.00%	47.50%	38.00%	27.00%	17.50%	10.50%	9.50%	10.50%
Mar2011	65.45%	43.18%	55.91%	56.82%	34.09%	27.73%	24.09%	10.45%	5.00%	13.18%
Apr2011	60.00%	52.00%	59.00%	50.00%	37.00%	26.50%	23.00%	14.00%	8.50%	13.50%
May2011	60.45%	46.82%	63.64%	54.09%	35.91%	26.82%	18.18%	10.91%	9.09%	16.36%
Jun2011	65.24%	47.62%	50.48%	56.19%	40.48%	27.62%	22.38%	12.38%	4.29%	11.43%
Jul2011	62.86%	49.05%	54.29%	51.43%	44.76%	28.10%	19.05%	13.81%	6.67%	14.29%
Aug2011	64.78%	44.78%	58.70%	52.61%	41.30%	25.65%	23.04%	13.48%	6.52%	12.61%
Sep2011	62.73%	46.36%	60.45%	55.00%	33.64%	26.36%	17.27%	11.82%	9.55%	11.36%
Oct2011	63.81%	46.67%	60.00%	57.14%	37.14%	27.62%	17.14%	12.86%	6.19%	12.38%
Nov2011	64.76%	48.57%	53.33%	53.33%	41.90%	33.33%	21.90%	9.52%	9.52%	11.90%
Dec2011	57.65%	46.47%	60.59%	55.88%	45.29%	24.71%	25.88%	12.35%	5.88%	12.35%
J

### Convert Carbon data to a DataFrame

Also creates a list of chart series names

In [102]:
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;

var df1 = DataFrame.LoadCsvFromString(chartXaxis +  strRet, '\t');

var chartSeriesNames = df1.Columns.Skip(1).Select(c => c.Name).ToArray();   // Skip first 1 which is the label "month"
chartSeriesNames

index,value
0,Brand1
1,Brand2
2,Brand3
3,Brand4
4,Brand5
5,Brand6
6,Brand7
7,Brand8
8,Brand9
9,Brand10


### Plotly.Net

API reference is https://plotly.net/reference/index.html

This produces an interactive inline chart. Click legend to toggle series on and off. Use Zoom and Pan to see detail.

In [103]:
using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;
using Microsoft.FSharp.Core;
using Microsoft.FSharp.Collections;

In [104]:

LinearAxis catAxis = new LinearAxis();
catAxis.SetValue("axistype","Category");
catAxis.SetValue("title", chartXaxis);
catAxis.SetValue("zerolinecolor", "#ffff");
catAxis.SetValue("gridcolor", "#ffff");
catAxis.SetValue("showline", true);
catAxis.SetValue("zerolinewidth",2);

In [105]:

LinearAxis yAxis = new LinearAxis();
yAxis.SetValue("title", "Respondents");
yAxis.SetValue("zerolinecolor", "#ffff");
yAxis.SetValue("gridcolor", "#ffff");
yAxis.SetValue("showline", true);
yAxis.SetValue("zerolinewidth",2);

Layout layout = new Layout();
layout.SetValue("xaxis", catAxis);
layout.SetValue("yaxis", yAxis);
layout.SetValue("title", "Data from Carbon");
layout.SetValue("plot_bgcolor", "#e5ecf6");
layout.SetValue("showlegend", true);

var traces = new List<Trace>();
for (int a = 0; a < chartSeriesNames.Length; a++)
{
    Trace t =new Trace("line");
    t.SetValue("x", df1[chartXaxis]); 
    t.SetValue("y", df1[chartSeriesNames[a]]); 
    t.SetValue("name",chartSeriesNames[a]); 
    traces.Add(t);
}

var fig = GenericChart.Figure.create(ListModule.OfSeq(traces),layout);
GenericChart.fromFigure(fig)

<!-- Plotly chart will be drawn inside this DIV -->

In [106]:
bool closed = engine.CloseJob();
Console.WriteLine($"Job closed = {closed} at {DateTime.Now:HH:mm:ss}");
int count = await engine.LogoutId("16499372");
Console.WriteLine($"Logout done. Licence borrow count is now {count}");

Job closed = True at 15:49:27
Logout done. Licence borrow count is now 1
